In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
forex_df = spark.read.format("delta").load("/mnt/forexratesdl/processed/forex_data")
forex_df.createOrReplaceTempView("tempviewforex")

In [0]:
forex_parkSQL = spark.sql("WITH cte AS (SELECT base_currency_id,target_currency_id,currency_name,amount,\
    LAG(amount) OVER(PARTITION BY base_currency_id,target_currency_id ORDER BY data_time) AS previus_amount,\
    ABS(amount - LAG(amount) OVER (PARTITION BY base_currency_id, target_currency_id ORDER BY data_time)) AS daily_change\
    FROM tempviewforex)\
    SELECT base_currency_id,target_currency_id,currency_name,\
    round(stddev(daily_change),3) AS volatility\
    FROM cte\
    GROUP BY base_currency_id, target_currency_id,currency_name\
    ORDER BY volatility desc;")

base_currency_id,target_currency_id,currency_name,volatility
USD,COP,Colombian Peso,17.984
USD,PYG,Paraguayan Guarani,10.757
USD,CRC,Costa Rican Colon,5.009
USD,ARS,Argentine Peso,2.061
USD,CLP,Chilean Peso,1.246
USD,DOP,Dominican Peso,0.236
USD,UYU,Uruguayan Peso,0.186
USD,NIO,Nicaraguan Cordoba,0.124
USD,VES,Venezuelan Bolovar Soberano,0.034
USD,PEN,Peruvian Nuevo Sol,0.028


In [0]:
forex_parkSQL.write.mode("append").format("delta").saveAsTable("presentation_forex_db.volatility")